<a href="https://colab.research.google.com/github/RebeccaRoberts/phd-codebites/blob/master/hierachical_sampling_for_dirichlet_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hierarchical Sampling for Dirichlet update from Polya

This is the sampling version of the 2D example that we use for ALBU and VB messages. 

In [17]:
def get_sampled_message(alphas=[1.2,0.8], p_W_given_Z=[0.8,0.2],N=20000,v=0):

  # Sample from the dirichlet
  a_samples = np.random.dirichlet(alphas, N)

  # Arrays for sampled categorical and its child
  Z_samples = np.zeros(len(a_samples))
  W_samples = np.zeros(len(a_samples))

  i = 0
  # Iterate over the samples
  for sample in a_samples:
    # Sample from the categorical Z
    Z_sample = np.random.choice(  
      a=list(range(len(sample))),  
      size=1,  
      p=sample 
    ) 
    # Save the sample
    Z_samples[i] = Z_sample
    
    # Sample from the child categorical, W, using Z_sample
    W_sample = np.random.choice(  
      a=[0, 1],  
      size=1,  
      p=p_W_given_Z[Z_sample][0] # its a matrix hence the [0]
    ) 
    # Save the sample
    W_samples[i] = W_sample
    i += 1

  # Reshape and stacking
  Z_samples = Z_samples.reshape(1,-1)
  W_samples = W_samples.reshape(1,-1)
  samples = np.hstack((Z_samples.T,W_samples.T))

  # Observe v (throw away the rest of the samples)
  samples = samples[samples[:,1] == v]

  # Total number of samples remaining
  sum_V = len(samples)

  # List of K categories for Z
  obs_list = list(range(len(alphas)))

  # Array to store the mixture of alpha and pk vaues
  pkak_array = np.zeros(len(alphas))
  i = 0
  for obs in obs_list:
    n_v = len(samples[samples[:,0] == obs])
    pkak_array[i] = n_v/sum_V
    i +=1
  return pkak_array

Z is of cardinality 3 (K = 3) and W of cardinality 2 (V = 2):

In [20]:
import numpy as np

# Number of samples
N = 200000

# Category of W that we observe
v = 1 # called v from word in vocab from LDA

# Dirichelt alpha parameters
alphas = np.array([0.8, 1.2, 2.0])

# Set up a conditional distribution: P(W|Z)
pW_Z1 = np.array([0.2, 0.8])
pW_Z2 = np.array([0.7, 0.3])
pW_Z3 = np.array([0.4, 0.6])
p_W_given_Z = np.vstack((pW_Z1,pW_Z2)) 
p_W_given_Z = np.vstack((p_W_given_Z,pW_Z3)) # stacked vertically for convenience


pkak_array = get_sampled_message(alphas=alphas, 
                                 p_W_given_Z=p_W_given_Z,
                                 N=N,
                                 v=v)


print("p_k a_k array from sampling:",pkak_array)

alphas_dash = alphas + pkak_array
print("sampling posterior:",alphas_dash)


# ALBU

# Calate the categorical coming back to the dirichlet
p_W_equals_v_given_Z = p_W_given_Z.T[v]
pZ_given_W_equals_v = p_W_equals_v_given_Z/sum(p_W_equals_v_given_Z)
pX = pZ_given_W_equals_v

# mixing alphas and incoming probs
pkak_array = alphas*pX;
pkak_array /= np.sum(pkak_array) # renormalize since it is only one observation
print("p_k a_k array from ALBU:",pkak_array)

# now add the original alphas to get the posterior dirichlet alphas at the polya node
alphas_dash = pkak_array + alphas

print('albu posterior:',alphas_dash)



p_k a_k array from sampling: [0.29169925 0.16362397 0.54467679]
sampling posterior: [1.09169925 1.36362397 2.54467679]
p_k a_k array from ALBU: [0.29090909 0.16363636 0.54545455]
albu posterior: [1.09090909 1.36363636 2.54545455]


2D example, easier to visualise:

In [18]:
# Category of W that we observe
v = 0 # called v from word in vocab from LDA

# Dirichelt alpha parameters
alphas = np.array([0.8, 1.2])

# Sample from the dirichlet
a_samples = np.random.dirichlet(alphas, N)

# Set up a conditional distribution: P(W|Z)
pW_Z1 = np.array([0.2, 0.8])
pW_Z2 = np.array([0.7, 0.3])
p_W_given_Z = np.vstack((pW_Z1,pW_Z2)) # stacked vertically for convenience

pkak_array = get_sampled_message(alphas=alphas, 
                                 p_W_given_Z=p_W_given_Z,
                                 N=N,
                                 v=v)


print("p_k a_k array from sampling:",pkak_array)

alphas_dash = alphas + pkak_array
print("sampling posterior:",alphas_dash)

# ALBU

# Calate the categorical coming back to the dirichlet
p_W_equals_v_given_Z = p_W_given_Z.T[v]
pZ_given_W_equals_v = p_W_equals_v_given_Z/sum(p_W_equals_v_given_Z)
pX = pZ_given_W_equals_v

# mixing alphas and incoming probs
pkak_array = alphas*pX;
pkak_array /= np.sum(pkak_array) # renormalize since it is only one observation
print("p_k a_k array from ALBU:",pkak_array)

# now add the original alphas to get the posterior dirichlet alphas at the polya node
alphas_dash = pkak_array + alphas

print('albu posterior:',alphas_dash)

p_k a_k array from sampling: [0.15925052 0.84074948]
sampling posterior: [0.95925052 2.04074948]
p_k a_k array from ALBU: [0.16 0.84]
albu posterior: [0.96 2.04]
